<a href="https://colab.research.google.com/github/reneebrecht/human-protein-atlas-image-classification/blob/f1_score/f1_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Get the best model

In [28]:
!pip install fsspec # this is needed for pandas
!pip install gcsfs # this is needed for access the cloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 7.3 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 271 kB 53.0 MB/s 
     |████████████████████████████████| 144 kB 57.2 MB/s 


In [29]:
from sklearn.metrics import f1_score
from google.colab import auth
import joblib
import os
import pandas as pd
import numpy as np



In [30]:
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = ' imposing-league-354107'
!gcloud config set project {project_id}

Updated property [core/project].


In [31]:
!mkdir -p /tmp/saved_model
!gsutil cp -r 'gs://human_proteins/saved_model' /tmp/saved_model

Copying gs://human_proteins/saved_model/Ada_Bea_0...
Copying gs://human_proteins/saved_model/Ada_Bea_1...
Copying gs://human_proteins/saved_model/Ada_Bea_10...
Copying gs://human_proteins/saved_model/Ada_Bea_11...
| [4 files][ 30.4 MiB/ 30.4 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://human_proteins/saved_model/Ada_Bea_12...
Copying gs://human_proteins/saved_model/Ada_Bea_13...
Copying gs://human_proteins/saved_model/Ada_Bea_14...
Copying gs://human_proteins/saved_model/Ada_Bea_15...
Copying gs://human_proteins/saved_model/Ada_Bea_16...
Copying gs://human_proteins/saved_model/Ada_Bea_17...
Copying gs://human_proteins/saved_model/Ada_Bea_18...
Copying gs://human_proteins/saved_model/Ada_Bea_19...
Copying gs://human_pr

In [41]:
df_score = pd.DataFrame(index=range(28))
for filename in os.listdir('/tmp/saved_model/saved_model'):
    f_path = os.path.join('/tmp/saved_model/saved_model', filename)
    try:
      if filename[0:2] != 'nn':
        my_model,y_pred, y_test, pipe = joblib.load(f_path)
      elif not 'model' in filename:
        y_pred, y_test = joblib.load(f_path)  
      else:
        continue
      model, loc_number = filename.rsplit('_', 1)
      if not model in df_score.columns:
        list_model = np.zeros(28)
        df_score[model] = list_model
      try:
        df_score.loc[df_score.index[int(loc_number)],model]= f1_score(y_test, y_pred).round(2)
      except ValueError:
        continue
    except IsADirectoryError:
      continue


In [42]:
df_score

,KNN,GradientBoostingClassifier,ExtraT_Bea,PAC_Bea,RiCcv_Bea,nn_5_layers_dropout,xgboostclass,BerNB_Bea,svc_rbf_C0_1,svc_rbf,Ada_Bea,RF_Bea
0,0.64,0.64,0.78,0.63,0.72,0.0,0.70,0.58,0.70,0.76,0.66,0.76
1,0.62,0.62,0.72,0.60,0.66,0.0,0.64,0.58,0.60,0.68,0.62,0.70
2,0.63,0.63,0.83,0.59,0.67,0.0,0.65,0.60,0.63,0.72,0.62,0.80
3,0.62,0.62,0.66,0.54,0.63,0.0,0.62,0.57,0.62,0.66,0.60,0.63
4,0.62,0.62,0.67,0.59,0.63,0.0,0.62,0.59,0.64,0.68,0.60,0.66
5,0.60,0.60,0.70,0.56,0.63,0.0,0.63,0.58,0.62,0.67,0.61,0.68
6,0.63,0.63,0.65,0.59,0.63,0.0,0.65,0.57,0.56,0.69,0.63,0.65
7,0.61,0.61,0.62,0.58,0.59,0.0,0.62,0.53,0.49,0.63,0.56,0.61
8,0.71,0.71,0.65,0.64,0.59,0.0,0.61,0.53,0.49,0.69,0.60,0.66
9,0.64,0.64,0.65,0.59,0.58,0.0,0.62,0.57,0.63,0.65,0.60,0.66


In [60]:
df_score.loc[len(df_score.index)] = [sum(df_score[model][0:28])/28 for model in df_score.columns]


In [62]:
from pathlib import Path  
filepath = Path('tmp/f1_scores.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_score.to_csv(filepath) 

In [63]:
!gsutil cp -r 'tmp/f1_scores.csv' gs://human_proteins/

Copying file://tmp/f1_scores.csv [Content-Type=text/csv]...
-
Operation completed over 1 objects/2.0 KiB.                                      
